In [2]:
import torch
import torch.nn as nn 
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

In [3]:
mnist_train=datasets.MNIST(root="./data_again",train=True,transform=ToTensor(),download=True)
mnist_test=datasets.MNIST(root="./data_again",train=False,transform=ToTensor(),download=True)

100%|██████████| 9912422/9912422 [00:04<00:00, 2030744.74it/s]


Extracting ./data_again\MNIST\raw\train-images-idx3-ubyte.gz to ./data_again\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 12683037.78it/s]


Extracting ./data_again\MNIST\raw\train-labels-idx1-ubyte.gz to ./data_again\MNIST\raw



100%|██████████| 1648877/1648877 [00:02<00:00, 817665.18it/s] 


Extracting ./data_again\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data_again\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data_again\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data_again\MNIST\raw



In [4]:
batchSize=4
train_data=DataLoader(mnist_train,batch_size=batchSize,shuffle=True)
test_data=DataLoader(mnist_test,batch_size=batchSize,shuffle=True)

In [7]:
class digitClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(784,100,bias=True),
                               nn.ReLU(),
                               nn.Linear(100,100,bias=True),
                               nn.ReLU(),
                               nn.Linear(100,10,bias=True))
    def forward(self,x):
        return self.net(x)

In [9]:
model=digitClassification()
lossfn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.001)

In [10]:
epochs=5
for epoch in range(epochs):
    model.train(True)
    totalloss=0.0
    for i,img in enumerate(train_data):
        optimizer.zero_grad()
        Ip,label=img
        predicted=model(Ip.view(batchSize,-1))
        loss=lossfn(predicted,label)
        loss.backward()
        optimizer.step()
    totalloss+=loss.item()
    avgLoss=(totalloss/len(train_data))*batchSize
    print(f"epoch {epoch}--->loss {avgLoss}")

epoch 0--->loss 0.0002460310141245524
epoch 1--->loss 2.8206332524617512e-05
epoch 2--->loss 0.00051276642481486
epoch 3--->loss 7.662545839945475e-05
epoch 4--->loss 6.613690853118897e-05


In [11]:
model.eval()

count=0
for _,testimage in enumerate(train_data):
    ip,op=testimage
    count+=1
    if count==1: break

with torch.no_grad():
    predicted=model(ip.view(batchSize,-1))
    _,predictedOutput=torch.max(predicted,dim=1)
    
#each number in the tensor (op/predictedOutput) correspinds to the class associated with the input image. 
print(f"actual label={op}") 
print(f"predicted label={predictedOutput}")

actual label=tensor([8, 0, 4, 3])
predicted label=tensor([8, 0, 4, 3])


In [19]:
correct=0
total=len(test_data)*batchSize
count=0
for _,vdata in enumerate(test_data):
    vip,vop=vdata
    vpred=model(vip.view(batchSize,-1))
    _,vclass=torch.max(vpred,dim=1)
    correct+=(vclass==vop).sum()
accuracy=(correct/total)*100
print(f"accuracy={accuracy}")   

accuracy=92.86000061035156
